<a href="https://colab.research.google.com/github/SeanMuInCa/learn_python/blob/master/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shivadumnawar/titanic-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/shivadumnawar/titanic-dataset/versions/1


In [16]:
import pandas as pd
# load the data
df = pd.read_csv(path + "/train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 选择特征
features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked']

df = df[features + ['Survived']].copy()  # 明确复制，避免 SettingWithCopyWarning

# 处理缺失值
df.loc[:, 'Age'] = df['Age'].fillna(df['Age'].median())  # 采用 `.loc[]` 赋值，避免警告
df.loc[:, 'Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

# 编码分类变量
df.loc[:, 'Sex'] = LabelEncoder().fit_transform(df['Sex'])  # 采用 `.loc[]` 赋值
df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)  # One-Hot 编码

# 划分数据集
X = df.drop(columns=['Survived'])
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [23]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

# 训练 Naïve Bayes 模型
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

# 5 折交叉验证
nb_scores = cross_val_score(nb_model, X_train, y_train, cv=5, scoring='accuracy')
print("Naïve Bayes 交叉验证平均准确率: ", nb_scores.mean())


Naïve Bayes 交叉验证平均准确率:  0.7752645161290322


In [24]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# 定义 SVM 训练参数
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# 网格搜索
grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# 最优 SVM 模型
best_svm = grid_search.best_estimator_
print("SVM 最优参数: ", grid_search.best_params_)

# 计算 SVM 交叉验证分数
svm_scores = cross_val_score(best_svm, X_train, y_train, cv=5, scoring='accuracy')
print("SVM 交叉验证平均准确率: ", svm_scores.mean())


SVM 最优参数:  {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
SVM 交叉验证平均准确率:  0.7849548387096774


In [25]:
from sklearn.metrics import accuracy_score, recall_score, f1_score

# 预测
y_pred_nb = nb_model.predict(X_test)
y_pred_svm = best_svm.predict(X_test)

# 计算评估指标
accuracy_nb = accuracy_score(y_test, y_pred_nb)
recall_nb = recall_score(y_test, y_pred_nb)
f1_nb = f1_score(y_test, y_pred_nb)

accuracy_svm = accuracy_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)

print(f"Naïve Bayes - Accuracy: {accuracy_nb:.2f}, Recall: {recall_nb:.2f}, F1 Score: {f1_nb:.2f}")
print(f"SVM - Accuracy: {accuracy_svm:.2f}, Recall: {recall_svm:.2f}, F1 Score: {f1_svm:.2f}")


Naïve Bayes - Accuracy: 0.79, Recall: 0.71, F1 Score: 0.73
SVM - Accuracy: 0.79, Recall: 0.70, F1 Score: 0.74
